In [ ]:
import os
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime
from sqlalchemy.orm import declarative_base, sessionmaker
from dotenv import load_dotenv
from langchain_cohere import ChatCohere
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from uuid import uuid4
from datetime import datetime
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Set up SQLite database
DATABASE_URL = "sqlite:///C:/Users/jarmo/gits/interface_app_once_a_day/backend/chat_history.db"
Base = declarative_base()
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Define ChatMessage model
class ChatMessage(Base):
    __tablename__ = "chat_messages"
    id = Column(Integer, primary_key=True, index=True)
    session_id = Column(String, index=True)
    role = Column(String)
    content = Column(Text)
    timestamp = Column(DateTime, default=datetime.utcnow)

# Create the database tables
Base.metadata.create_all(bind=engine)

# Function to get a new database session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Function to save chat message to the database
def save_chat_message(session_id, role, content, db):
    db_message = ChatMessage(session_id=session_id, role=role, content=content)
    db.add(db_message)
    db.commit()
    db.refresh(db_message)
    return db_message

# Function to get chat message history from the database
def get_chat_message_history(session_id, db):
    return db.query(ChatMessage).filter(ChatMessage.session_id == session_id).all()

# Custom ChatMessageHistory class to manage chat history
class ChatMessageHistory:
    def __init__(self, messages=None):
        if messages is None:
            messages = []
        self.messages = messages

    def add_messages(self, new_messages):
        self.messages.extend(new_messages)

    def get_messages(self):
        return self.messages

# Initialize Cohere client with API key from environment variables
llm = ChatCohere(
    cohere_api_key=os.getenv("COHERE_API_KEY")
)

csv_analyzer = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert in CSV analysis. You will analyze a CSV file and return summary statistics. You will ask for the file path and return the summary statistics.",
        ),
        ("human", "{input}"),

    ]
) | llm | StrOutputParser()

# Define tools for the agent to use
tools = [
    Tool.from_function(
        name="CSV Analyzer",
        description="Analyze a CSV file and return summary statistics. You will ask for the file path and return the summary statistics.",
        input_schema={"file_path": str},
        func=csv_analyzer.invoke,
    ),     
]

# Pull agent prompt from hub
agent_prompt = hub.pull("hwchase17/react-chat")
# Create the agent using the pulled prompt and tools
agent = create_react_agent(llm, tools, agent_prompt)
# Create an executor for the agent
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Function to get chat message history for a session
def get_memory(session_id):
    db = next(get_db())
    messages = get_chat_message_history(session_id=session_id, db=db)
    # Convert database messages to a format suitable for ChatMessageHistory
    formatted_messages = [{"role": msg.role, "content": msg.content} for msg in messages]
    return ChatMessageHistory(formatted_messages)

# Create chat agent with message history
chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",

)

# Generate a unique session ID
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

# Main loop to handle user input and generate responses
while (q := input("> ")) != "exit":
    db = next(get_db())
    save_chat_message(SESSION_ID, "human", q, db)
    
    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID, "message_history": get_memory(SESSION_ID)}},
    )
    
    save_chat_message(SESSION_ID, "system", response["output"], db)
    print("")
    print("user:", q)
    print("")
    print("system:", response["output"])


c:\Users\jarmo\anaconda3\envs\learn_langchain-env\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Session ID: 9d23754c-2741-4b35-a84c-6b30d7faa4a3

user: analyze csv

system: I'm ready to analyze your CSV file! Please provide the file path or upload the file, and I'll get started. I'll provide a comprehensive summary of the data, including column details, data types, and basic statistics for numerical columns. If there are specific areas you'd like to focus on, feel free to guide me accordingly!

user: C:\Users\jarmo\gits\interface_app_once_a_day\titanic.csv

system: I apologize for the inconvenience. I am unable to access local file paths. To proceed with the analysis, please provide the content of the CSV file directly in your request, or upload the file to a cloud-based storage service and share the public link. I will be happy to assist you once I have access to the data.

user: ok nyt

system: I'm sorry, I didn't understand your request. Could you please provide more details or clarify your question? If you need assistance with a CSV file, I can help analyze and discuss its co